In [1]:
import tensorflow as tf
import numpy as np

In [93]:
import logging
logging.getLogger('tensorflow').setLevel(logging.DEBUG)

In [3]:
import codecs
import json
from tensorflow.contrib import rnn

In [30]:
# Hyper-parameters
hidden_size   = 100  # hidden layer's size
seq_length    = 25   # number of steps to unroll
learning_rate = 1e-1
vocab_size = 51
batch_size = 128

In [5]:
import vocab
vocab_ = vocab.VocabBuilder('ptb.char.train.txt')

In [87]:
def model_fn(features,labels,mode,params,config):
    #size of x - [time_step,batch_size,vocab_size]
    #create a LSTMcell of size hidden_size
    x = features['x']
    print(x.shape)
    #unstack in time steps (that is the 2nd dimension)
    x = tf.unstack(x,seq_length,1)
    rnn_cell = rnn.BasicLSTMCell(hidden_size)
    outputs, states = rnn.static_rnn(cell=rnn_cell,inputs=x,dtype=tf.float64)
    #create variables for softmax weight and bias
    softmax_w = tf.get_variable(name='softmax_w', shape=[hidden_size,vocab_size],dtype=tf.float64,initializer=tf.initializers.truncated_normal())
    softmax_b = tf.get_variable(name='softmax_b', shape=[1,vocab_size],dtype=tf.float64,initializer=tf.initializers.zeros())
    
    output_sequence_logits = []
    for output in outputs:
        output_sequence_logits.append(tf.matmul(output,softmax_w) + softmax_b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_sequence_logits,labels=labels))
    print(loss)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optmizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        specs = dict(mode=mode,loss=loss,train_op=optmizer.minimize(loss))
    
    return tf.estimator.EstimatorSpec(**specs)
    

In [75]:
def input_generator(filename,seq_len):
    def generator():
        with open(filename, 'r') as f:
            char_list = f.read()
            clist=[]
            for i in range(0,len(char_list),2):
                clist.append(char_list[i])
            p=0
            while p+seq_len+1<=len(clist):
                input_ = clist[p:p+seq_len]
                output_ = clist[p+1:p+1+seq_len]
                p = p+seq_len
                yield dict(x=vocab_.getOneHot(input_),y=vocab_.getOneHot(output_))
    return generator

In [90]:
from tensorflow.contrib.learn.python.learn.learn_io.generator_io import generator_input_fn
train_input_fn = generator_input_fn(x=input_generator('ptb.char.train.txt',seq_length),target_key='y',shuffle=False,batch_size=batch_size,queue_capacity = 10*batch_size,num_epochs=None)

In [91]:
model = tf.estimator.Estimator(model_fn=model_fn,model_dir="./BasicLSTM")

In [97]:
#model.train(input_fn=train_input_fn,steps=1)